In [29]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Initialize Spark Session
spark = SparkSession.builder.appName("SpotifyDataNormalization").getOrCreate()

# Read data into a dataframe
df = spark.read.option("multiline", "true").json("all_recent_tracks.json")

# Convert timestamp to normal date and time 
df = df.withColumn("played_at", F.from_unixtime(F.col("played_at") / 1000, "yyyy-MM-dd HH:mm:ss"))

# Overwrite the CSV file, save as a single file
df.coalesce(1).write.option("header", "true").mode("overwrite").csv("clean_tracks.csv")

# Show the transformed 'played_at' column
df.select("played_at").show(truncate=False)


+-------------------+
|played_at          |
+-------------------+
|2025-03-29 01:30:46|
|2025-03-29 01:32:49|
|2025-03-29 01:35:48|
|2025-03-29 01:37:49|
|2025-03-29 01:42:10|
|2025-03-29 01:45:10|
|2025-03-29 01:48:15|
|2025-03-29 01:51:23|
|2025-03-29 01:54:55|
|2025-03-29 01:57:53|
|2025-03-29 02:00:28|
|2025-03-29 02:02:31|
|2025-03-29 02:05:23|
|2025-03-29 02:07:52|
|2025-03-29 02:10:10|
|2025-03-29 02:12:11|
|2025-03-29 02:16:11|
|2025-03-29 02:19:38|
|2025-03-29 02:22:25|
|2025-03-29 02:25:22|
+-------------------+
only showing top 20 rows



In [30]:
""" import shutil
# Move the part file to desired location
part_file = "clean_tracks.csv\part-00000-e767b6d4-f389-44a7-91b0-dec958759f38-c000.csv"
destination = "clean_recent_tracks.csv"
shutil.move(part_file,destination) """


<>:1: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\p'
C:\Users\IFEOMA\AppData\Local\Temp\ipykernel_9276\441510881.py:1: SyntaxWarning: invalid escape sequence '\p'
  """ import shutil


' import shutil\n# Move the part file to desired location\npart_file = "clean_tracks.csv\\part-00000-e767b6d4-f389-44a7-91b0-dec958759f38-c000.csv"\ndestination = "clean_recent_tracks.csv"\nshutil.move(part_file,destination) '